In [ ]:
import sys
from dotenv import load_dotenv
import os

load_dotenv()

PROJECT_ROOT = os.getenv("PROJECT_ROOT")
sys.path.append(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "src"))
sys.path.append(os.path.join(PROJECT_ROOT, "src/scripts"))
sys.path.append(os.path.join(PROJECT_ROOT, "src/utils"))

# Now import the module
from model_full_finetune_2 import *

==((====))==  Unsloth 2025.9.5: Fast Gemma3_Text patching. Transformers: 4.56.1. vLLM: 0.10.2.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.663 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


Filter: 100%|██████████| 100/100 [00:00<00:00, 449.24 examples/s]


In [28]:
args = TrainingArguments(
	per_device_train_batch_size=12,
	per_device_eval_batch_size=8,
	gradient_accumulation_steps=24,
	warmup_ratio=0.05,
	num_train_epochs=5,
	learning_rate=2e-5,
	fp16=not is_bfloat16_supported(),
	bf16=is_bfloat16_supported(),
	logging_steps=10,
	optim="adamw_torch",
	weight_decay=0.01,
	lr_scheduler_type="linear",
	seed=3407,
	output_dir=os.path.join(MODELS_PATH, "full_finetuned_gemma-3-270m-it"),
	report_to="tensorboard",
	eval_strategy="steps",
	eval_steps=10,
	save_strategy="steps",
	save_steps=10,
	save_total_limit=1,
	metric_for_best_model="eval_loss",
	greater_is_better=False,
)

In [29]:
trainer = SFTTrainer(
	model=model,
	tokenizer=tokenizer,
	train_dataset=train_dataset,
	eval_dataset=val_dataset,
	dataset_text_field="text",
	max_seq_length=MAX_SEQ_LENGTH,
	dataset_num_proc=2,
	packing=True,
	args=args,
	max_grad_norm=1.0,
)

# trainer = train_on_responses_only(
# 	trainer,
# 	instruction_part="<start_of_turn>user\n",
# 	response_part="<start_of_turn>model\n",
# )


Unsloth: Tokenizing ["text"] (num_proc=20): 100%|██████████| 100/100 [00:06<00:00, 15.79 examples/s]


In [30]:
import torch
# GPU stats and training call
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.663 GB.
21.449 GB of memory reserved.


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 13,042 | Num Epochs = 5 | Total steps = 230
O^O/ \_/ \    Batch size per device = 12 | Gradient accumulation steps = 24
\        /    Data Parallel GPUs = 1 | Total batch size (12 x 24 x 1) = 288
 "-____-"     Trainable parameters = 268,098,176 of 268,098,176 (100.00% trained)


Step,Training Loss,Validation Loss
10,2.099000,1.899153
20,1.786600,1.692840
30,1.663000,1.632922


Unsloth: Not an error, but Gemma3TextModel does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
